In [1]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, confusion_matrix

from sklearn.svm import SVC 
from pipelines import preprocessor, preprocessor2
from preprocess import extra_features, separate

In [2]:
train=pd.read_csv('Datasets/train.csv')
train.head(2)

<IPython.core.display.Javascript object>

,Tour_ID,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,...,package_accomodation,package_food,package_transport_tz,package_sightseeing,package_guided_tour,package_insurance,night_mainland,night_zanzibar,first_trip_tz,cost_category
0,tour_id1hffseyw,ITALY,45-64,With Children,0.0,2.0,Visiting Friends and Relatives,Beach Tourism,"Friends, relatives",Package Tour,...,Yes,Yes,Yes,No,No,No,0,7,Yes,High Cost
1,tour_idnacd7zag,UNITED KINGDOM,25-44,With Spouse,1.0,1.0,Leisure and Holidays,Wildlife Tourism,"Travel agent, tour operator",Package Tour,...,Yes,Yes,Yes,No,No,No,0,7,Yes,High Cost


In [3]:
separate(train)

Target Column: ['cost_category']

Numerical Columns: ['night_mainland', 'night_zanzibar']

Categorical Columns: ['age_group', 'travel_with', 'tour_arrangement', 'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz', 'package_sightseeing', 'package_guided_tour', 'package_insurance', 'first_trip_tz']

Text Columns: ['country', 'purpose', 'main_activity', 'info_source']


In [4]:
X=preprocessor2(train)

In [5]:
X.shape

(18506, 34)

In [6]:
y=train['cost_category']

le=LabelEncoder()
y=le.fit_transform(y)
y

array([0, 0, 1, ..., 1, 1, 3])

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X, y, shuffle=True, random_state=1)

In [8]:
kfold=StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

for fold, (train_idx,val_idx) in enumerate(kfold.split(X,y)):
        X_train, X_val= X[train_idx], X[val_idx]
        y_train, y_val=y[train_idx], y[val_idx]

In [25]:
cl=SVC(gamma="auto", kernel='rbf')
cl.fit(X_train, y_train)

SVC(gamma='auto', kernel='linear')

In [26]:
y_pred=cl.predict(X_val)

In [28]:
print(classification_report(y_val, y_pred, zero_division=True, target_names=le.classes_))

              precision    recall  f1-score   support

   High Cost       0.43      0.43      0.43       367
 Higher Cost       0.64      0.74      0.69       487
Highest Cost       1.00      0.00      0.00        36
    Low Cost       1.00      0.00      0.00       156
  Lower Cost       0.29      0.01      0.02       257
 Normal Cost       0.44      0.74      0.55       547

    accuracy                           0.50      1850
   macro avg       0.63      0.32      0.28      1850
weighted avg       0.53      0.50      0.43      1850



In [ ]:
clf=SVC()

C_values = [0.1, 1, 10, 100, 1000]
for C in C_values:
    clf.C = C
    clf.fit(X_train, y_train)
    print("C = {}: accuracy = {}".format(C, clf.score(X_val, y_val)))

# Tune the kernel parameter
kernels = [ "poly", "linear", "rbf"]
for kernel in kernels:
    clf.kernel = kernel
    clf.fit(X_train, y_train)
    print("Kernel = {}: accuracy = {}".format(kernel, clf.score(X_val, y_val)))

# Tune the decision_function_shape parameter
decision_function_shapes = ["ovo", "ovr"]
for decision_function_shape in decision_function_shapes:
    clf.decision_function_shape = decision_function_shape
    clf.fit(X_train, y_train)
    print("Decision_function_shape = {}: accuracy = {}".format(decision_function_shape, clf.score(X_val, y_test)))

# Tune the class_weight parameter
class_weights = {0: 1, 1: 2}
clf.class_weight = class_weights
clf.fit(X_train, y_train)
print("Class_weight = {}: accuracy = {}".format(class_weights, clf.score(X_val, y_val)))

C = 0.1: accuracy = 0.42486486486486486
C = 1: accuracy = 0.4745945945945946
C = 10: accuracy = 0.4859459459459459
C = 100: accuracy = 0.5151351351351351
C = 1000: accuracy = 0.5172972972972973
Kernel = poly: accuracy = 0.4962162162162162
